In [1]:
# -*- coding: utf-8 -*-
import h5py
import os
import pickle
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

In [2]:
import argparse
args = argparse.Namespace(mri_sequences='ABDEFKT', data_sequences='TBAKDEF')
ckpt = '/net/scratch/hanliu/radiology/prostatex/results/wang-fold-bal/z4ktwfw9/checkpoints/epoch=102-valid_acc=0.81.ckpt' #splendid-night-1029
from wang_args import WangClassifier
model = WangClassifier.load_from_checkpoint(ckpt, **vars(args))
model.eval()
model.w_ensemble


  | Name   | Type       | Params
--------------------------------------
0 | conv   | ModuleList | 30.8 K
1 | linear | ModuleList | 29.6 K
2 | fusion | Sequential | 28.8 K
--------------------------------------
89.2 K    Trainable params
0         Non-trainable params
89.2 K    Total params


tensor([0.0000, 0.3425, 0.0000, 0.2357, 0.0000, 0.0000, 0.0000, 0.4218])

In [73]:
transf = transforms.ToTensor()
def batch_predict(images):
    b = torch.cat(tuple(transf(i).unsqueeze(0) for i in images), 0)
    return torch.sigmoid(model(b[:, model.mri_index])).matmul(model.w_ensemble).detach().numpy()


In [45]:
dataset = torchvision.datasets.DatasetFolder('tbakd3_npy/uint8/5folds/4/train_bal', extensions='npy', loader=np.load, transform=transforms.ToTensor())
images, labels = torch.cat([x[0].unsqueeze(0) for x in dataset], 0), np.array([x[1] for x in dataset])
images = np.moveaxis(images.numpy(), 1, -1)

In [74]:
batch_predict(images[:10])

array([0.20120803, 0.59290636, 0.51603764, 0.29302526, 0.25249836,
       0.2920285 , 0.3032642 , 0.3405705 , 0.28027856, 0.5537454 ],
      dtype=float32)

In [72]:
from lime import lime_image


In [75]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(images[0]), 
                                         batch_predict, # classification function
                                         top_labels=1, 
                                         hide_color=0, 
                                         num_samples=1000) # number of images that will be sent to classification function

ValueError: Input array must have a shape == (..., 3)), got (64, 64, 7)